# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Получение общей информации

In [1]:
from collections import Counter

import pandas as pd
from pymystem3 import Mystem

In [2]:
customer_solvency = pd.read_csv('/datasets/data.csv')

In [3]:
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
customer_solvency.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Описание данных:**

- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита.

**Вывод:** таблица со статистикой платежеспособности клиентов содержит 21525 строк и 12 столбцов. Колонки `days_employed` и `total_income` содержат строки с пропущенными значениями, которые необходимо обработать. Таблица содержит 3 типа данных: float64, int64, object, для удобства анализа нужно изменить тип данных с вещественного на целочисленный. Названия столбцов понятны и читаемы, написаны в "змеином_регистре" и на одном языке.

## Предобработка данных

### Обработка пропусков

С помощью метода isna() посчитаю количество пропущенных значений в каждой колонке:

In [6]:
customer_solvency.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод:** из 12 столбцов пропуски присутствуют в двух: `days_employed` и `total_income`. 
Важно отметить, что данные в этих столбцах - количественные, а значит пропуски можно заполнить характерными значениями выборки.

Я думаю, что одинаковое количество пропусков в этих столбцах могло появиться из-за того, что 2174 человека не "устроены официально" и не могут предоставить данные об трудовом стаже в днях, поскольку нет записи в трудовой книжке, отсюда и отсутствие данных в колонке с ежемесячным доходом, так как эти параметры напрямую взаимосвязаны.

Я заметила, что в столбце `days_employed` некоторые значения сильно отличаются, а именно, имеют шестизначный порядок значений (например, значение 340266 в строке с 4 индексом соответствует стажу в 932 года (340266/365), что говорит о некорректно заполненных данных.

Некорректно введенные значениями могут повлиять на вычисление среднего выборки для заполнения пропущенных значений.
Поскольку я не могу получить информацию от того, кто заполнял эти данные, буду придерживаться следующего: приму, что максимальный трудовой стаж в днях может быть 55 лет, а именно 365*55 = 20075 дней и определю с помощью логической индексации, много ли строк встречается с некорректно заполненным стажем.

In [7]:
customer_solvency[customer_solvency['days_employed'] >= 20075]['days_employed'].count()

3445

Поскольку таких строк достаточно много, проверю как изменяется медиана для каждой категории заемщиков. Так же столбец `days_employed` имеет отрицательные значения, которые необходимо привести к абсолютным.

In [8]:
customer_solvency['days_employed'] = customer_solvency['days_employed'].abs()
days_employed_median = (
    customer_solvency.groupby('income_type')['days_employed'].median().astype('int')
)
days_employed_median

income_type
безработный        366413
в декрете            3296
госслужащий          2689
компаньон            1547
пенсионер          365213
предприниматель       520
сотрудник            1574
студент               578
Name: days_employed, dtype: int64

Медиана сильно различается для каждой категории, поэтому для заполнения пропусков я буду использовать медиану, вычисленную отдельно для каждой группы.

In [9]:
customer_solvency['days_employed'] = (
    customer_solvency['days_employed']
    .fillna(customer_solvency.groupby('income_type')['days_employed'].transform('median'))
)
customer_solvency.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Пропуски в столбце `total_income` заменю более детально, а именно медианным значением для каждой группы.

In [10]:
total_income_median = (
    customer_solvency.groupby('income_type')['total_income'].median().astype('int')
)
total_income_median

income_type
безработный        131339
в декрете           53829
госслужащий        150447
компаньон          172357
пенсионер          118514
предприниматель    499163
сотрудник          142594
студент             98201
Name: total_income, dtype: int64

In [11]:
customer_solvency['total_income'] = (
    customer_solvency['total_income']
    .fillna(customer_solvency.groupby('income_type')['total_income'].transform('median'))
)
customer_solvency.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Так же в данных я заметила, что в колонке `dob_years` некоторые значения принимают ноль. Я решила проверить, сколько таких нулей присутствуют в колонке с возрастом. Так как возраст таковым быть не может, буду считать это за пропущенные данные, которые можно восстановить. 

Данные в этой колонке количественные, поэтому пропущенные значения заполню средним арифметическим выборки.

In [12]:
customer_solvency[customer_solvency['dob_years'] == 0]['dob_years'].count()

101

In [13]:
# среднее арифметическое выборки для столбца dob_years
dob_years_mean = int(customer_solvency['dob_years'].mean())
dob_years_mean

43

In [14]:
# замена нулевых значений на среднее арифметическое
customer_solvency['dob_years'] = customer_solvency['dob_years'].replace(0, dob_years_mean)

Осуществлю проверку с помощью логической индексации:

In [15]:
customer_solvency[customer_solvency['dob_years'] == 0]['dob_years'].count()

0

После обработки пропусков проверяю, все ли пропущенные значения удалось удалить:

In [16]:
customer_solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод:** пропущенные значения в данных - частая практика в работе аналитика. Пропуски могут возникать по некоторым причинам: механический фактор - некорректная работа при выгрузке данных; человеческий фактор - ошибки в наборе данных. Поскольку в исходном датафрейме пропущенные значения неважны для целей исследования, но в тех же строках присутствуют ценные данные, то можно заменить пропущенные значения другими. 

Для двух типов значений - категориальные и количественные существуют свои методики работы с пропусками. В датафрейме данные в колонках `days_employed`, `dob_years` и `total_income` - количественные, поэтому пропуски в таких значениях можно заменять характерными значениями выборки - средним арифметическим или медианой.

Таким образом, на этапе обработки пропущенных значений мне удалось заменить пропущенные данные в трех столбцах: `days_employed`, `dob_years`, `total_income` для дальнейшего анализа датафрейма `customer_solvency`. 

### Замена типов данных

Чтобы изменить тип данных в столбцах `days_employed` и `total_income` с вещественного на целочисленный воспользуюсь методом astype(). 

In [17]:
customer_solvency[['days_employed', 'total_income']] = (
    customer_solvency[['days_employed', 'total_income']].astype('int')
)
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод:** с точки зрения анализа данных, достаточно целочисленных значений, которые выглядят более читаемо и их проще анализировать.

### Обработка дубликатов

Проверю датафрейм на наличие явных дубликатов с помощью метода duplicated():

In [18]:
customer_solvency.duplicated().sum()

54

От явных дубликатов избавляюсь методом drop_duplicates():

In [19]:
customer_solvency = customer_solvency.drop_duplicates().reset_index(drop=True)

Осуществляю проверку:

In [20]:
customer_solvency.duplicated().sum()

0

От явных дубликатов избавилась. Теперь перехожу к неявным.

На наличие неявных дубликатов проверяю категориальные столбцы с помощью метода unique():

In [21]:
for column in ['education', 'family_status', 'income_type', 'purpose']:
    print(column, customer_solvency[column].unique(), sep=': ', end='\n\n')

education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

purpose: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимо

В столбце `education` символы имеют разный регистр, из-за чего значения дублируются. В столбце `family_status` дублирующихся значений нет, но не все значения приведены к нижнему регистру, это нужно исправить, иначе в дальнейшем можно допустить ошибку при наборе данных.

Все значения в данных столбцах приведу к нижнему регистру методом str.lower():

In [22]:
customer_solvency['family_status'] = customer_solvency['family_status'].str.lower()

In [23]:
customer_solvency['education'] = customer_solvency['education'].str.lower()

Проверю, какие уникальные значения остались в столбцах `education` и `family_status`:

In [24]:
for column in ['education', 'family_status']:
    print(column, customer_solvency[column].unique(), sep=': ', end='\n\n')

education: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']

family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']



Посмотрю какие значения присутствуют в столбце `children`:

In [25]:
customer_solvency['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В данном столбце встречаются аномальные значения, "-1" и "20". Сначала приведу все значения к абсолютным:

In [26]:
customer_solvency['children'] = customer_solvency['children'].abs()

In [27]:
# проверка столбца 'children' на количество аномальных значений
customer_solvency[customer_solvency['children'] == 20]['children'].count()

76

Встречающееся число "20" в данных о количестве детей - аномальное значение, ошибка человека, который заполнял эти данные. Так как в условии задачи стоит вопрос "зависит ли количество детей на факт погашения кредита в срок", то данные в столбце 'children' важны и влияют на выводы исследования. Так как аномальные значения встречаются в 76 строках - это 0,35% ото всех данных датафрейма, я принимаю решение удалить строки с искаженными данными.

In [28]:
customer_solvency = (
    customer_solvency.drop(customer_solvency[customer_solvency['children'] == 20].index)
    .reset_index(drop=True)
)

Выведу общую информацию о датафрейме после обработки дубликатов.

In [29]:
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21395 entries, 0 to 21394
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21395 non-null  int64 
 1   days_employed     21395 non-null  int64 
 2   dob_years         21395 non-null  int64 
 3   education         21395 non-null  object
 4   education_id      21395 non-null  int64 
 5   family_status     21395 non-null  object
 6   family_status_id  21395 non-null  int64 
 7   gender            21395 non-null  object
 8   income_type       21395 non-null  object
 9   debt              21395 non-null  int64 
 10  total_income      21395 non-null  int64 
 11  purpose           21395 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод:** так же как и пропуски, наличие дубликатов - часто встречаемое явление при работе с данными. Дублирующиеся значения могут появляться по вине человека, который заполнил данные (случайно включенный capslock, редактирование строк с помощью копирования). Анализ данных при большом количестве дублирующихся значений недостоверен. Уметь находить дубликаты и избавляться от них - важный навык аналитика. 

Дубликаты подразделяются на явные - полностью повторяющиеся строки в датафрейме, и неявные - те, где значения в строках могут иметь одинаковых смысл, но отличаться регистром или быть написаны на разных языках. Обнаружить явные дубликаты помогает метод duplicated(), а избавиться от них можно с помощью метода drop_duplicates(), от неявных - методом replace(), а с помощью ручного поиска приводить символы в названиях к нижнему регистру методом str.lower(). 

Таким образом, мне удалось избавиться от дубликатов с помощью вышеописанных методов.

### Лемматизация

В датафрейме нужно провести лемматизацию столбца `purpose`. Для этого импортирую библиотеку с функцией лемматизации на русском языке - pymystem3. Напишу функцию lemmatize(), которая для переданного текста будет возвращать список лемм.

In [30]:
stem = Mystem()

def lemmatize(text):
    lemmas = stem.lemmatize(text)
    return lemmas

Чтобы выделить категории, необходимо найти наиболее встречающиеся леммы, для этого использую контейнер Counter из модуля collections. Так как нужно лемматизировать значения столбца `purpose`, то предварительно необходимо этот столбец склеить в одну строку. 

In [31]:
lemmas = lemmatize(' '.join(customer_solvency['purpose']))
Counter(lemmas)

Counter({'покупка': 5881,
         ' ': 54866,
         'жилье': 4446,
         'приобретение': 461,
         'автомобиль': 4292,
         'дополнительный': 903,
         'образование': 3999,
         'сыграть': 765,
         'свадьба': 2326,
         'операция': 2594,
         'с': 2905,
         'на': 2217,
         'проведение': 769,
         'для': 1288,
         'семья': 637,
         'недвижимость': 6332,
         'коммерческий': 1305,
         'жилой': 1228,
         'строительство': 1872,
         'собственный': 629,
         'подержать': 844,
         'свой': 2227,
         'со': 626,
         'заниматься': 904,
         'сделка': 937,
         'получение': 1312,
         'высокий': 1368,
         'подержанный': 110,
         'профильный': 434,
         'сдача': 651,
         'ремонт': 605,
         '\n': 1})

**Вывод:** лемматизация позволила мне выделить конкретные категории цели получения кредита у заемщиков: *автомобиль, образование, свадьба, недвижимость.*

### Категоризация данных

Проведу категоризацию - объединю данные в категории. Для этого применю метод qcut() к столбцу `total_income` для определения границ ежемесячного дохода.

In [32]:
pd.qcut(customer_solvency['total_income'], q=4, precision=0)

0        (195750.0, 2265604.0]
1         (107626.0, 142594.0]
2         (142594.0, 195750.0]
3        (195750.0, 2265604.0]
4         (142594.0, 195750.0]
                 ...          
21390    (195750.0, 2265604.0]
21391     (142594.0, 195750.0]
21392      (20666.0, 107626.0]
21393    (195750.0, 2265604.0]
21394      (20666.0, 107626.0]
Name: total_income, Length: 21395, dtype: category
Categories (4, interval[float64]): [(20666.0, 107626.0] < (107626.0, 142594.0] < (142594.0, 195750.0] < (195750.0, 2265604.0]]

**1 категория - "по уровню дохода"**

Распределю клиентов по четырем категориям: 
- клиенты с ежемесячным доходом `менее 107626` попадают в категорию `низкий`; 
- клиенты с доходом `107626 - 142594` - категория `средний`;
- клиенты с доходом `142594 - 195750` - категория `выше среднего`;
- клиенты с доходом `больше 195750` - категория `высокий`.

In [33]:
income_labels = ['низкий', 'средний', 'выше среднего', 'высокий']
customer_solvency['income_category'] = (
    pd.qcut(customer_solvency['total_income'], q=4, labels=income_labels, precision=0)
)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,выше среднего


**2 категория - "по целям кредита"**

Распределю клиентов по четырем категориям: 
- клиенты с целью взять кредит на `автомобиль`; 
- клиенты с целью взять кредит на `образование`;
- клиенты с целью взять кредит на `свадьбу`;
- клиенты с целью взять кредит для операций с `недвижимостью`.

Функция calculate_purpose_category() принимает данные с описанием цели кредита, проводит их лемматизацию, на выходе определяет категорию по наличию ключевого слова в леммах.

In [34]:
def calculate_purpose_category(purpose):
    lemmas = set(lemmatize(purpose))
    
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif {'недвижимость', 'жилье'} & lemmas:
        return 'недвижимость'
    
    return 'не определено'

customer_solvency['purpose_category'] = customer_solvency['purpose'].apply(calculate_purpose_category)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высокий,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,выше среднего,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,высокий,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,выше среднего,свадьба


Осуществляю проверку, чтобы убедиться, что все категории по целям кредита определены:

In [35]:
customer_solvency[customer_solvency['purpose_category'] == 'не определено']['purpose_category'].count() 

0

**3 категория - "по количеству детей"**

Распределю клиентов по шести категориям: 
- клиенты, у которых нет детей, попадают в категорию 0; 
- клиенты, у которых есть один ребенок, попадают в категорию 1;
- клиенты, у которых два ребенка, - категория 2;
- клиенты, у которых три ребенка, - категория 3;
- клиенты, у которых четыре ребенка, - категория 4;
- клиенты, у которых пять детей, - категория 5.

В исходном датафрейме есть столбец `children`, по которому можно проводить категоризацию, поэтому новый столбец добавлять не нужно.

**4 категория - "по уровню образования"**

- высшее - категория 0;
- среднее - категория 1;
- неоконченное высшее - категория 2;
- начальное - категория 3;
- ученая степень - категория 4.

В исходном датафрейме существует столбец с категориями клиентов по уровню образования, поэтому новую колонку добавлять не нужно.

**5 категория - "по семейному положению"**

- женат / замужем - категория 0;
- гражданский брак - категория 1;
- вдовец / вдова - категория 2;
- в разводе - категория 3;
- не женат / не замужем - категория 4.

В исходном датафрейме существует столбец с категориями клиентов по семейному положению, поэтому новую колонку добавлять не нужно.

Для оптимального хранения информации о 4 и 5 категориях вынесу их в отдельные таблицы `education_categories` и `family_status_categories`:

In [36]:
education_categories = customer_solvency[['education', 'education_id']]
education_categories = education_categories.drop_duplicates().reset_index(drop=True)
education_categories

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [37]:
family_status_categories = customer_solvency[['family_status', 'family_status_id']]
family_status_categories = (
    family_status_categories.drop_duplicates().reset_index(drop=True)
)
family_status_categories

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Из датафрейма `customer_solvency` удалю колонки с именами категорий:

In [38]:
customer_solvency = customer_solvency.drop(['education', 'family_status'], axis=1)
customer_solvency.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,высокий,недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,средний,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,выше среднего,недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,высокий,образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,выше среднего,свадьба


**Вывод:** категоризация позволяет сгруппировать данные для их последующего анализа. 

Я разделила данные по пяти категориям:

- *по уровню дохода*;
- *по целям кредита*;
- *по количеству детей*;
- *по уровню образования*;
- *по семейному положению*.

Сгруппированные данные позволяют находить зависимости между категориями и возвратом кредита в срок.

## Интерпретация полученных результатов

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
df_example = customer_solvency.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во клиентов', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во клиентов,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14107,1063,7.54%,92.46%
1,4856,445,9.16%,90.84%
2,2052,194,9.45%,90.55%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


Можно сделать два **вывода:**
- клиенты без детей реже допускают задолжность по кредиту (примерно на 10% в сравнении с заемщиками с детьми);
- процент должников среди людей с детьми не зависит от их количества.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
debt_by_family_status = (
    customer_solvency.pivot_table(index='family_status_id', values='debt', aggfunc='mean')
)
debt_by_family_status['debt'] = round(debt_by_family_status['debt'] * 100, 2)
debt_by_family_status = (
    family_status_categories.merge(debt_by_family_status, on='family_status_id', how='left')
)
debt_by_family_status

,family_status,family_status_id,debt
0,женат / замужем,0,7.55
1,гражданский брак,1,9.27
2,вдовец / вдова,2,6.60
3,в разводе,3,7.04
4,не женат / не замужем,4,9.75


**Вывод:** семейное положение влияет на факт погашения кредита в срок. Заемщики с семейным положением `не женат / не замужем`, `гражданский брак` чаще других категорий допускают задолжность по кредиту. Возможно, это связано с их меньшей социальной ответственностью.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
debt_by_income_category = (
    customer_solvency.pivot_table(index='income_category', values='debt', aggfunc='mean')
)
debt_by_income_category['debt'] = round(debt_by_income_category['debt'] * 100, 2)
debt_by_income_category

,debt
income_category,
низкий,7.98
средний,8.77
выше среднего,8.52
высокий,7.12


**Вывод:** категория клиентов по ежемесячному доходу уровня `средний` и `выше среднего` чаще не возвращают кредит в срок. Интересный вывод, что категории по уровню дохода `низкий` и `высокий` реже допускают задолжности по кредиту в сравнении с категорией `средний`. Могу предположить, что людям со `средним` доходом и доходом `выше среднего` сложнее планировать свой бюждет.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
debt_by_purpose_category = (
    customer_solvency.pivot_table(index='purpose_category', values='debt', aggfunc='mean')
)
debt_by_purpose_category['debt'] = round(debt_by_purpose_category['debt'] * 100, 2)
debt_by_purpose_category

,debt
purpose_category,
автомобиль,9.34
недвижимость,7.24
образование,9.23
свадьба,7.87



По полученным значениям можно сделать **вывод**, что цель взятия кредита по операциям с недвижимостью является наименее рисковой для банка, так как заемщики с данной целью реже всего допускают просрочки по кредиту.

Категория клиентов по целям кредита `автомобиль` и `образование` в равной степени чаще допускают задолжности.

## Общий вывод

Проделана большая работа по анализу данных со статистикой о платежеспособности клиентов. Передо мной стоял вопрос: влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?

Для того, чтобы ответить на поставленный вопрос я проделала несколько этапов по работе с данными, а именно:
- обработала пропущенные значения;
- заменила типы данных;
- обработала дубликаты;
- провела лемматизацию;
- категоризовала данные.

На основании проведенного анализа данных банку можно дать следующие рекомендации:

- поскольку клиенты без детей реже допускают просрочку по кредиту, то стоит тщательнее проверять семьи с детьми;

- клиенты с семейным положением `не женат / не замужем` так же требуют более внимательной проверки;

- при принятии решении об одобрении кредита банком размер ежемесячного дохода клиента не должен быть решающим фактором;

- кредит на недвижимость является более безопасной сделкой для банка в сравнении с остальными категориями.